# Homework Assignment 4

## Topics: Variable engineering

In [14]:
import boto3
import pandas as pd
import numpy as np
from scipy.stats import norm
import statsmodels.api as smf
from sklearn.linear_model import LogisticRegression

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## 1. Created new s3 bucket: data-448-bucket-callaghan
## Using pandas to read the data files

file_key = 'train.csv'
file_key2 = 'test.csv'

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')

train = pd.read_csv(file_content_stream, sep = '|')
test = pd.read_csv(file_content_stream2, sep = '|')

train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [27]:
X.head(1)

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
0,5,1054,54.7,7,0,3,0.027514,0.051898,0.241379


In [31]:
## 2. Using the train data-frame to engineer at least 7 different features that can help to predict fraud
## using the given input variables.



## Finding significance of input variables with logistic regression

## Defining the input and target variables
X = train.drop(columns = ['fraud'])
Y = train['fraud']

## Building the model
logit_md = LogisticRegression(max_iter = 10000).fit(X, Y)

## Function that returns the p-values for each variable
def logit_pvalue(model, x):
   
    p1 = model.predict_proba(x)
    n1 = len(p1)
    m1 = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    answ = np.zeros((m1, m1))
    for i in range(n1):
        answ = answ + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p1[i,1] * p1[i, 0]
    vcov = np.linalg.inv(np.matrix(answ))
    se = np.sqrt(np.diag(vcov))
    t1 =  coefs/se  
    p1 = (1 - norm.cdf(abs(t1))) * 2
    return p1



sd_model = smf.Logit(Y, smf.add_constant(X)).fit(disp=0)
print(sd_model.pvalues)
sd_model.summary()

const                        7.633935e-01
trustLevel                   1.905754e-14
totalScanTimeInSeconds       2.471198e-03
grandTotal                   1.855095e-03
lineItemVoids                9.779393e-14
scansWithoutRegistration     2.576972e-07
quantityModifications        9.222598e-01
scannedLineItemsPerSecond    1.094202e-01
valuePerSecond               1.055505e-02
lineItemVoidsPerPosition     5.690744e-13
dtype: float64


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  fraud   No. Observations:                 1879
Model:                          Logit   Df Residuals:                     1869
Method:                           MLE   Df Model:                            9
Date:                Wed, 16 Feb 2022   Pseudo R-squ.:                  0.8378
Time:                        01:40:42   Log-Likelihood:                -65.204
converged:                       True   LL-Null:                       -402.05
Covariance Type:            nonrobust   LLR p-value:                3.111e-139
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.3834      1.274     -0.301      0.763      -2.879       2.113
trustLevel                   -6.2374      0.815     -7.657      0.000      -7.834      -4.641
totalScanTimeInSeconds        0.0024      0.001      3.027      0.002       0.001       0.004
grandTotal                    0.0403      0.013      3.113      0.002       0.015       0.066
lineItemVoids                 3.1560      0.424      7.444      0.000       2.325       3.987
scansWithoutRegistration      0.4824      0.094      5.152      0.000       0.299       0.666
quantityModifications         0.0133      0.136      0.098      0.922      -0.254       0.281
scannedLineItemsPerSecond     5.7116      3.568      1.601      0.109      -1.281      12.705
valuePerSecond              -23.6513      9.249     -2.557      0.011     -41.780      -5.523
lineItemVoidsPerPosition    -64.8158      8.993     -7.208      0.000     -82.441     -47.191
=============================================================================================

Possibly complete quasi-separation: A fraction 0.82 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [ ]:
## (Variable 1) lowTrust - when the trustLevel score is one or two
train['lowTrust'] = np.where(train['trustLevel'] <= 2, 1, 0)

In [ ]:
## Engineer the same features on the test data-frame